Making A Simple Neural Network In PyTorch then comparing it to a neural network that will be made from scratch.

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input):
        # Convolution layer C1: 1 input image channel, 6 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a Tensor with size (N, 6, 28, 28), where N is the size of the batch
        c1 = F.relu(self.conv1(input))
        # Subsampling layer S2: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 6, 14, 14) Tensor
        s2 = F.max_pool2d(c1, (2, 2))
        # Convolution layer C3: 6 input channels, 16 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a (N, 16, 10, 10) Tensor
        c3 = F.relu(self.conv2(s2))
        # Subsampling layer S4: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 16, 5, 5) Tensor
        s4 = F.max_pool2d(c3, 2)
        # Flatten operation: purely functional, outputs a (N, 400) Tensor
        s4 = torch.flatten(s4, 1)
        # Fully connected layer F5: (N, 400) Tensor input,
        # and outputs a (N, 120) Tensor, it uses RELU activation function
        f5 = F.relu(self.fc1(s4))
        # Fully connected layer F6: (N, 120) Tensor input,
        # and outputs a (N, 84) Tensor, it uses RELU activation function
        f6 = F.relu(self.fc2(f5))
        # Gaussian layer OUTPUT: (N, 84) Tensor input, and
        # outputs a (N, 10) Tensor
        output = self.fc3(f6)
        return output

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### Neural Network From Scratch

Here we will build the fundamentals of Neural Networks from scratch, starting with the perceptron.

### Perceptrons

The perceptron approximates a single neuron with n binary inputs. The perceptron computes a weighted sum of its inputs and "fires" if that weighted sum is zero or greater.

In [ ]:
#define the perceptron step function
def step_function(x):
    return 1 if x >= 0 else 0

In [ ]:
#build dot product function that will be used
def dot(v, w):
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

In [ ]:
def perceptron_output(weights, bias, x):
    """
    returns 1 if the perceptron 'fires' and 0 if not
    """
    calculation = dot(weights, x) + bias
    return step_function(calculation)

### Feed-Forward Neural Networks

There are obvious limitations to a simple perceptron so in order to perform more complicated solutions we incorporate what are called "feed-forward neural networks". These networks consist of discrete layers of neurons, each connected to the next and where each layer feeds into the next layer.

These networks are incorporated using the following layers
- input layer: receives inputs and feeds them forward unchanged
- hidden layer (could be multiple): consists of neurons that take the outputs of the previous layer, performs some calculations and passes the result to the next layer
- output layer: produces the final outputs

Feed-Forward networks also implement a smooth approximation of the step function within perceptrons, this smooth approximation is known as the sigmoid function. This is because in order to train these networks we must implement some calculus and therefore we need continuous functions. The sigmoid function can also be thought of as the logistic function.

In [ ]:
import math

#sigmoid function
def sigmoid(t):
    return 1 / (1 + math.exp(-t))

Therefore in order to calculate the output of our neurons, we use:

In [ ]:
def neuron_output(weights, inputs):
    return sigmoid(dot(weights, inputs))

Therefore we can think of a neuron and represent it as a list of weights whose length is one more than the number of inputs to that neuron (to account for the bias term).

Then we can represent a neural "network" as a list of layers, where each layer is just a list of the neurons in that layer.